# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## To do
- Analyse directional specific results
    - Lossless vs baseline vs optimum
    - yaw values
- Assess difference in speed of calculation and optimal values when using different fidelity for final optimisation
- Increase number of turbines

## Setup

In [1]:
# import libraries
import logging
import os

import numpy as np
import xarray as xr
from pqdm.processes import pqdm
from tqdm.notebook import tqdm

import utils

logging.basicConfig(
    format="%(asctime)s:%(levelname)s:%(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

## Baseline values

In [2]:
# extract probabilities for full wind speed/direction range
_, Sector_frequency, P = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=utils.WS_DEFAULT,
    wd=utils.WD_DEFAULT,
)

09:14:08:WARNING:P renormalised as total probability was 0.9997679498494154
09:14:08:WARNING:no reference simulation provided


In [3]:
# drop wind speeds below cut in speed
ind_cut_in = np.argmax(utils.wfm_low.windTurbines.power(utils.WS_DEFAULT) > 0)
ws = utils.WS_DEFAULT[ind_cut_in:]

In [4]:
# run baseline simulations
print("--- Lossless ---")
sim_res_ref_lossless, _, _ = utils.run_sim(
    wfm=utils.wfm_lossless,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=ws,
    wd=utils.WD_DEFAULT,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)

print("\n--- Low Fidelity ---")
sim_res_ref_low, _, _ = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=ws,
    wd=utils.WD_DEFAULT,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)

print("\n--- High Fidelity ---")
sim_res_ref_high, _, _ = utils.run_sim(
    wfm=utils.wfm_high,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=ws,
    wd=utils.WD_DEFAULT,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)

09:14:08:WARNING:no reference simulation provided
09:14:08:WARNING:no reference simulation provided


--- Lossless ---
Annual energy [GWh]: 82.049
LCoE [USD/MWh]: 42.918
Capacity factor [%]: 52.000

--- Low Fidelity ---
Annual energy [GWh]: 79.157
LCoE [USD/MWh]: 44.487
Capacity factor [%]: 50.167

--- High Fidelity ---


09:14:08:WARNING:no reference simulation provided


Annual energy [GWh]: 79.500
LCoE [USD/MWh]: 44.295
Capacity factor [%]: 50.384


## Optimise across all wind directions

In [5]:
# initialise optimal yaw dataset
coords = {
    "wt": list(sim_res_ref_low.wt.values),
    "wd": list(sim_res_ref_low.wd.values),
    "ws": list(sim_res_ref_low.ws.values),
}
yaw_opt = xr.Dataset(
    data_vars={
        "init": (
            list(coords.keys()),
            np.full([len(x) for x in coords.values()], np.nan),
        ),
        "final": (
            list(coords.keys()),
            np.full([len(x) for x in coords.values()], np.nan),
        ),
    },
    coords=coords,
)

In [6]:
# run optimisations and save output
simulations_in = [
    dict(
        wd=wd_,
        sim_res_ref_low=sim_res_ref_low,
        sim_res_ref_high=sim_res_ref_high,
        Sector_frequency=Sector_frequency,
        P=P,
    )
    for wd_ in sim_res_ref_low.wd.values
]
simulations_out = pqdm(
    array=simulations_in,
    function=utils.optimise_direction,
    n_jobs=int(0.75 * os.cpu_count()),
    argument_type="kwargs",
)
for input, (yaw_opt_init, yaw_opt_final) in tqdm(
    zip(simulations_in, simulations_out),
    total=len(simulations_out),
    desc="Saving values to dataset",
):
    yaw_opt["init"].loc[
        :,
        input["wd"],
        :,
    ] = yaw_opt_init
    yaw_opt["final"].loc[
        :,
        input["wd"],
        :,
    ] = yaw_opt_final

QUEUEING TASKS | :   0%|          | 0/24 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/24 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/24 [00:00<?, ?it/s]

Saving values to dataset:   0%|          | 0/24 [00:00<?, ?it/s]

In [7]:
# rerun simulation for optimum
sim_res_opt, _, _ = utils.run_sim(
    wfm=utils.wfm_high,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=yaw_opt.final,
    ws=ws,
    wd=yaw_opt.wd,
    sim_res_ref=sim_res_ref_high,
    Sector_frequency=Sector_frequency,
    P=P,
)
sim_res_opt

<xarray.SimulationResult>
Dimensions:            (wt: 9, wd: 24, ws: 22)
Coordinates:
  * ws                 (ws) float64 4.0 5.0 6.0 7.0 8.0 ... 22.0 23.0 24.0 25.0
  * wd                 (wd) int64 0 15 30 45 60 75 ... 270 285 300 315 330 345
  * wt                 (wt) int64 0 1 2 3 4 5 6 7 8
    type               (wt) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Data variables: (12/26)
    WS_eff             (wt, wd, ws) float64 4.0 5.001 6.001 ... 23.92 24.93
    TI_eff             (wt, wd, ws) float64 0.1 0.1 0.1 ... 0.1042 0.1038 0.1034
    Power              (wt, wd, ws) float64 6.374e+04 1.487e+05 ... 2e+06 2e+06
    CT                 (wt, wd, ws) float64 0.7659 0.787 ... 0.06056 0.05352
    tilt               int64 0
    yaw                (wt, wd, ws) float64 8.454 8.986 8.986 ... 0.1334 0.1334
    ...                 ...
    lcoe               (wd, wt) float64 54.74 57.56 56.79 ... 46.73 48.45 48.08
    cap_fac            (wt, wd) float64 0.4077 0.4167 0.465 ... 0.4564 0.4641
    lcoe_direction     (wd) float64 56.35 53.63 48.05 ... 38.42 46.83 47.78
    lcoe_overall       float64 44.11
    cap_fac_direction  (wd) float64 0.3961 0.4161 0.4645 ... 0.4766 0.4671
    cap_fac_overall    float64 0.506

## View overall results of optimisation

In [8]:
# display comaprison of optimum to baseline
print("--- LCoE ---")
print(f"Lossless [USD/MWh] : {sim_res_ref_lossless.lcoe_overall.values:.3f}")
print(f"Baseline [USD/MWh] : {sim_res_ref_high.lcoe_overall.values:.3f}")
print(f"Optimum [USD/MWh]  : {sim_res_opt.lcoe_overall.values:.3f}")
print(
    f"Recovered [%]      : {100-100*(sim_res_ref_lossless.lcoe_overall.values - sim_res_opt.lcoe_overall.values)/(sim_res_ref_lossless.lcoe_overall.values - sim_res_ref_high.lcoe_overall.values):.2f}"
)
print("\n--- Capacity Factor ---")
print(f"Lossless [%]  : {100*sim_res_ref_lossless.cap_fac_overall.values:.3f}")
print(f"Baseline [%]  : {100*sim_res_ref_high.cap_fac_overall.values:.3f}")
print(f"Optimum [%]   : {100*sim_res_opt.cap_fac_overall.values:.3f}")
print(
    f"Recovered [%] : {100-100*(sim_res_ref_lossless.cap_fac_overall.values - sim_res_opt.cap_fac_overall.values)/(sim_res_ref_lossless.cap_fac_overall.values - sim_res_ref_high.cap_fac_overall.values):.2f}"
)

--- LCoE ---
Lossless [USD/MWh] : 42.918
Baseline [USD/MWh] : 44.295
Optimum [USD/MWh]  : 44.109
Recovered [%]      : 13.50

--- Capacity Factor ---
Lossless [%]  : 52.000
Baseline [%]  : 50.384
Optimum [%]   : 50.596
Recovered [%] : 13.13
